<a href="https://colab.research.google.com/github/AliButtar/Cognyne/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Most Recent One (Suggested)
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-6r5vsjmh
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-6r5vsjmh
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8310 sha256=ba75563554a71784b159367db190033b12ed79b6f88c42bf53c5ab363e393e37
  Stored in directory: /tmp/pip-ephem-wheel-cache-tx905vo9/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [ ]:
!pip show keras-vggface

Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, keras, pyyaml, pillow, six, scipy, h5py
Required-by: 


In [ ]:
!pip install keras_applications

     |████████████████████████████████| 51kB 3.7MB/s 


In [ ]:
import keras_vggface
# print version
print(keras_vggface.__version__)

0.6


In [ ]:
pip install mtcnn

     |████████████████████████████████| 2.3MB 8.3MB/s 


In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import numpy as np
import cv2
import csv
from google.colab.patches import cv2_imshow
import os

In [ ]:
# extract a single face from a given photograph
def extract_face(pixels, required_size=(224, 224)):
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    print(results)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    
    return face_array

In [ ]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(img):
    # extract faces
    face = [extract_face(f) for f in img]
    
    print(face)
    
    # convert into an array of samples
    sample = asarray(face, 'float32')
    
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(sample, version=2)
    
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    
    # perform prediction
    yhat = model.predict(sample)

    return yhat

In [ ]:
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        return True
    else:
        return False

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
def run_webcam():
  take_photo()

  frame = cv2.imread('photo.jpg')
  
  img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  
  e = get_embeddings([img])
  
  return e, frame

In [ ]:
# df = pd.DataFrame(columns=['Name','Position', 'ID', 'Password', "Embeddings"])
# df.to_csv('user.csv', index = False)

In [ ]:
def signup():
    
    e, frame = run_webcam()
    
    df = pd.read_csv("user.csv") 
        
    
    have_account = False 
    for arr in df['Embeddings']:
        arr = [float(x) for x in arr[1:-1].split(',')]
        if(is_match(arr, e[0])):
            have_account = True
                         
    if (have_account):
        print("You already have an account.")
        signin(frame, e, True)
        
    else:
        name = input("Enter your Name: ")
        position = input("Enter your Position: ")
        userId = input("Enter you Registration number or Employee id: ")
        password = input("Enter you Password: ")
        
        
        df = df.append({'Name': name,'Position': position, 'ID': userId, 'Password': password}, ignore_index=True)
        df.at[df.shape[0]-1, 'Embeddings'] = [emb for emb in e[0]]
        
        df.to_csv('user.csv', index = False)
            
        print("You have signed up!")

In [ ]:
def signin(frame = None, e = None, redirected = False):

    global NAME
    global ID
    global E

    if(not (redirected)):
        e, frame = run_webcam()

    E=e

    df = pd.read_csv("user.csv") 

    index = -1
    i = 0
    for arr in df['Embeddings']:
        arr = [float(x) for x in arr[1:-1].split(',')]
        if(is_match(arr, e)):
            index = i
            break
        i = i+1

    if (index != -1):
        user_info = df.loc[index]
        NAME = user_info['Name'] 
        ID = user_info['ID']
        print("Successfully signed in")
        # Write some text on the frame.
        cv2.putText(frame, "Name " + user_info['Name'], (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)

        # Write some more text on the frame at different location.
        cv2.putText(frame, "ID: "+ str(user_info['ID']), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)

        cv2_imshow(frame)

    else:
        print("You donot have an account")
        

In [ ]:
signup()

<IPython.core.display.Javascript object>

[{'box': [225, 138, 109, 137], 'confidence': 0.9998643398284912, 'keypoints': {'left_eye': (269, 182), 'right_eye': (318, 196), 'nose': (297, 205), 'mouth_left': (261, 234), 'mouth_right': (308, 245)}}]
[array([[[236, 237, 221],
        [236, 237, 221],
        [237, 236, 221],
        ...,
        [237, 237, 225],
        [238, 238, 226],
        [238, 238, 226]],

       [[236, 237, 221],
        [236, 237, 221],
        [237, 236, 221],
        ...,
        [237, 237, 225],
        [238, 238, 226],
        [237, 237, 225]],

       [[236, 237, 221],
        [236, 237, 221],
        [237, 236, 221],
        ...,
        [236, 236, 224],
        [237, 237, 225],
        [237, 237, 225]],

       ...,

       [[222, 225, 216],
        [222, 225, 216],
        [224, 224, 214],
        ...,
        [ 32,  35,  40],
        [ 30,  33,  38],
        [ 29,  32,  37]],

       [[221, 225, 216],
        [224, 227, 218],
        [233, 234, 224],
        ...,
        [ 29,  32,  37],
        [ 

In [ ]:
signin()

In [ ]:
NAME, ID, E

('Muhammad Ali Buttar',
 75,
 array([[0.        , 2.13911   , 1.133177  , ..., 0.        , 0.01169913,
         0.00977477]], dtype=float32))

In [ ]:
# test = pd.DataFrame(columns=["Class_name", "Embeddings"])
# test.to_csv('class.csv')
# df = pd.DataFrame(columns=["Students_Names",'Students_ID'])
# df.to_csv('ML-01.csv')

In [ ]:
def create_class():
    df = pd.read_csv("class.csv", index_col = 0) 

    class_name = input('Enter the name of your Class: ')

    count = df.shape[0]
    
    df.at[count, 'Class_name'] = class_name
    df.at[count, 'Embeddings'] = [emb for emb in E[0]]

    df.to_csv('class.csv')

    print("Class created successfully!")

In [ ]:
create_class()

Enter the name of your Class: ML4
Class created successfully!


In [ ]:
def join_class(class_name):

    print('Show Teacher Face')

    e, _ = run_webcam()

    df = pd.read_csv("class.csv", index_col = 0) 

    index = -1
    i = 0
    for cl in df['Class_name']:
        if ( cl == class_name ):
            index = i
            break
        i = i+1

    if (index != -1):
        em = df['Embeddings'][index]
        em = [float(x) for x in em[1:-1].split(',')]
        if (is_match(e, em)):
            print("Show your own face.")
            e2, _ = run_webcam()

            if (is_match(e2, E)):

                if not os.path.exists(class_name+'.csv'):
                  
                  df_class = pd.DataFrame(columns=["Students_Names",'Students_ID'])
                  df_class.to_csv(class_name+'.csv')

                class_df = pd.read_csv(class_name + ".csv", index_col = 0) 
                class_df = class_df.append({'Students_Names': NAME, 'Students_ID': ID}, ignore_index=True)
                class_df.to_csv(class_name + ".csv")
                print(class_name + " Class Joined")

            else:
                print("Signin with your own account.")
        
        else:
            print("Incorrect Teacher!")

In [ ]:
df = pd.read_csv('class.csv', index_col=0)
df

,Class_name,Embeddings
0,BSSE-7C,"[0.0, 0.20406674, 2.698341, 2.2890139, 1.00103..."
1,ML-01,"[0.0, 0.20406674, 2.698341, 2.2890139, 1.00103..."
2,ML,"[0.0, 2.13911, 1.133177, 9.5343275, 0.4623918,..."
3,ML2,"[0.0, 2.13911, 1.133177, 9.5343275, 0.4623918,..."
4,ML3,"[0.0, 1.3310406, 2.0747457, 11.555553, 2.74519..."
5,ML4,"[0.0, 0.0, 0.0, 0.0, 0.34843898, 0.0, 0.0, 0.0..."


In [ ]:
join_class("ML-01")